In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as opt

In [2]:
class Leaf:
    def __init__(self,value):
        self.value = value

In [3]:
class Node:
    def __init__(self,branches,threshold):
        self.branches = branches
        self.threshold = threshold
        
    def get(self,x):
        return self.branches[0] if x < self.threshold else self.branches[1]
        

In [4]:
class Tree:
    def __init__(self,root):
        self.root = root
        
    def predict(self,x):
        item = self.root
        while isinstance(item,Node):
            item = item.get(x)
        return item

In [5]:
r=Node([Leaf('young'),Leaf('old')],18)
t=Tree(r)
print(t.predict(2).value)

young


In [6]:
df=pd.read_csv("iris.csv")

In [7]:
print(t.predict(20).value)

old


In [ ]:
class CART:
    def __init__(self,df,y_name,X_names):
        self.df = df
        self.y_name = y_name
        self.X_names = X_names
        self.tree = None
        
    def create_tree(self):
        root = self._node_or_leaf(self.df)
        self.tree = Tree(root)
        return self.tree
    
    def _gini_impurity(self, df):
        unique, counts = np.unique(df[self.y_name].values, return_counts=True)
        N = df[self.y_name].values.ravel().size
        p = counts/N
        print(unique)
        print(p)
        return 1. - np.sum(p**2)
    
    def _opt_fun(self,df,split_name):
        df1 = df[df[split_name]<split_threshold]
        
    def _node_or_leaf(self,df):
        loss_parent = 0.99
        loss_best, split_df, split_threshold, split_name = self._loss_best(df)
        if loss_best < loss_parent:
            branches = []
            for i in range(2):
                branches.append(self._node_or_leaf(split_df[i]))
            item = Node(branches,split_threshold)
            print(f"Node({loss_best})")
        else:
            value = np.unique(df[self.y_name])[0]
            item = Leaf(value)
            print(f"Leaf({value})")
        return item
    
    def _loss_best(self,df):
        loss0 = 1.1
        for name in self.X_names:
            split_threshold_ = np.median(df[name].values)
            split_df_ = [df[df[name]<split_threshold_],
                        df[df[name]>=split_threshold_]]
            loss = self._loss(split_df_[0]) + self._loss(split_df_[1])
            if loss < loss0:
                loss0 = loss
                split_threshold = split_threshold_
                split_df = split_df_
                split_name = name
                
            print(loss0)
                
        return loss0, split_df, split_threshold, split_name
    
    def _loss(self,df):
        return self._gini_impurity(df)
            
        
        

In [ ]:
df.columns
X_names=["sepal_length","sepal_width"]
df[X_names]

In [ ]:
c = CART(df,"species",X_names)
c.create_tree()